# Land stress impact factors for pasture

Exiobase categories:
- Permanent pastures – Grazing-Cattle
- Permanent pastures – Grazing-Meat animals
- Permanent pastures – Grazing-Raw milk

LC-Impact stressors
- Land stress: Pasture

In [ ]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments from json file
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [ ]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=arguments["exio_11_path"])
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [ ]:
# aggregate relevant drivers
groups = exio3_11.satellite.get_index(
    as_dict=True,
    grouping_pattern=arguments["exiobase_grouping_patterns"]["land_pasture"]
)

exio3_11.satellite_agg = exio3_11.satellite.copy(new_name="Aggregated land stress - pasture")

for df_name, df in zip(exio3_11.satellite_agg.get_DataFrame(data=False, with_unit=True, with_population=False),
exio3_11.satellite_agg.get_DataFrame(data=True, with_unit=True, with_population=False)):
    if df_name == "unit":
        exio3_11.satellite_agg.__dict__[df_name] = df.groupby(groups).apply(lambda x: " & ".join(x.unit.unique()))
    else:
        exio3_11.satellite_agg.__dict__[df_name] = df.groupby(groups).sum()

In [4]:
# diagonalize and calculate land stress for pasture
et1_diag = exio3_11.satellite_agg.diag_stressor(("Land stress - pasture"))
exio3_11.et1_diag = et1_diag
exio3_11.calc_all()

/nix/store/7wbnyr3241jd0p97l6db02skmsc7cy8z-python3.12-pymrio/lib/python3.12/site-packages/pymrio/core/mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/nix/store/7wbnyr3241jd0p97l6db02skmsc7cy8z-python3.12-pymrio/lib/python3.12/site-packages/pymrio/core/mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/nix/store/7wbnyr3241jd0p97l6db02skmsc7cy8z-python3.12-pymrio/lib/python3.12/site-packages/pymrio/core/mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/nix/store/7wbnyr3241jd0p97l6db02skmsc7cy8z-python3.12-pymrio/lib/python3.12/site-packages/pymrio/core/mriosystem.p

In [5]:
exio3_11.et1_diag.D_cba # consumption based distribution of the driver i.e DR origin

region                                                           AT        \
sector                                                   Paddy rice Wheat   
region sector                                                               
AT     Paddy rice                                               0.0   0.0   
       Wheat                                                    0.0   0.0   
       Cereal grains nec                                        0.0   0.0   
       Vegetables, fruit, nuts                                  0.0   0.0   
       Oil seeds                                                0.0   0.0   
...                                                             ...   ...   
WM     Membership organisation services n.e.c. (91)             0.0   0.0   
       Recreational, cultural and sporting services (92)        0.0   0.0   
       Other services (93)                                      0.0   0.0   
       Private households with employed persons (95)            0.0   0.0   
       Extra-territorial organizations and bodies               0.0   0.0   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                                      0.0   
       Wheat                                                           0.0   
       Cereal grains nec                                               0.0   
       Vegetables, fruit, nuts                                         0.0   
       Oil seeds                                                       0.0   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)                    0.0   
       Recreational, cultural and sporting services (92)               0.0   
       Other services (93)                                             0.0   
       Private households with employed persons (95)                   0.0   
       Extra-territorial organizations and bodies                      0.0   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                            0.0   
       Wheat                                                                 0.0   
       Cereal grains nec                                                     0.0   
       Vegetables, fruit, nuts                                               0.0   
       Oil seeds                                                             0.0   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                          0.0   
       Recreational, cultural and sporting services (92)                     0.0   
       Other services (93)                                                   0.0   
       Private households with employed persons (95)                         0.0   
       Extra-territorial organizations and bodies                            0.0   

region                                                              \
sector                                                   Oil seeds   
region sector                                                        
AT     Paddy rice                                              0.0   
       Wheat                                                   0.0   
       Cereal grains nec                                       0.0   
       Vegetables, fruit, nuts                                 0.0   
       Oil seeds                                               0.0   
...                                                            ...   
WM     Membership org

## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [6]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in exio3_11.et1_diag.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s = pd.DataFrame(columns)
dr_s

AT        \
                                                         Paddy rice Wheat   
region sector                                                               
AT     Paddy rice                                               0.0   0.0   
       Wheat                                                    0.0   0.0   
       Cereal grains nec                                        0.0   0.0   
       Vegetables, fruit, nuts                                  0.0   0.0   
       Oil seeds                                                0.0   0.0   
...                                                             ...   ...   
WM     Membership organisation services n.e.c. (91)             0.0   0.0   
       Recreational, cultural and sporting services (92)        0.0   0.0   
       Other services (93)                                      0.0   0.0   
       Private households with employed persons (95)            0.0   0.0   
       Extra-territorial organizations and bodies               0.0   0.0   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                                      0.0   
       Wheat                                                           0.0   
       Cereal grains nec                                               0.0   
       Vegetables, fruit, nuts                                         0.0   
       Oil seeds                                                       0.0   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)                    0.0   
       Recreational, cultural and sporting services (92)               0.0   
       Other services (93)                                             0.0   
       Private households with employed persons (95)                   0.0   
       Extra-territorial organizations and bodies                      0.0   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                            0.0   
       Wheat                                                                 0.0   
       Cereal grains nec                                                     0.0   
       Vegetables, fruit, nuts                                               0.0   
       Oil seeds                                                             0.0   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                          0.0   
       Recreational, cultural and sporting services (92)                     0.0   
       Other services (93)                                                   0.0   
       Private households with employed persons (95)                         0.0   
       Extra-territorial organizations and bodies                            0.0   

                                                                    \
                                                         Oil seeds   
region sector                                                        
AT     Paddy rice                                              0.0   
       Wheat                                                   0.0   
       Cereal grains nec                                       0.0   
       Vegetables, fruit, nuts                                 0.0   
       Oil seeds                                               0.0   
...                                                            ...   
WM     Membership organisation services n.e.c. (91)            0.0   
       Recreatio

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [ ]:
# harmonize regions 

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# harmonize regions between lc-impact and exiobase
all_row_region_keys = list(row_eu_countries.keys()) + list(row_asia_pacific_countries.keys()) + list(row_african_countries.keys()) + list(row_american_countries.keys()) + list(row_middle_eastern_countries.keys())
exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
print(len(exio_regions_without_row)) # exiobase contains 44 exact regions + 5 row regions

# combine exact regions from exiobase with exact row regions
all_regions = list(all_row_region_keys) + list(exio_regions_without_row)
print("Regions: ", len(all_regions))
print("Unique regions: ", len(set(all_regions)))
assert len(all_regions) == len(set(all_regions)), "There are duplicate regions in the all_regions list"

In [ ]:
# prepare lc-impact
lci = pd.read_excel(arguments["lc_impact_path"] + "/11-Land stress/CFs_land_Use_average.xlsx", sheet_name="occupation average country", skiprows=1, header=[0,1])
# Flatten the multi-level columns and rename them
lci.columns = [' '.join(col).strip() for col in lci.columns]
# rename first column to Country
lci.rename(columns={lci.columns[0]: "Country"}, inplace=True)
lci = lci[["Country", "Pasture Median"]]
lci

In [9]:
# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Country code not found for ", name)
            return None  # Return None if country not found

extra_mappings = {
    "Turkey": "TR",
    "Russia": "RU",
    "Bahamas, The": "BS",
    "Brunei": "BN",
    "Cape Verde": "CV",
    "Congo DRC": "CD",
    "Falkland Islands": "FK",
    "Gambia, The": "GM",
    "Ivory Coast": "CI",
    "The Former Yugoslav Republic of Macedonia": "MK",
    "Myanmar (Burma)": "MM",
    "Reunion": "RE",
    "St. Vincent and the Grenadines": "VC",
    "Svalbard": "SJ",
    "Swaziland": "SZ",
    "US Virgin Islands": "VI",
    "Western Samoa": "WS",
}

lci["Country_Code"] = lci["Country"].apply(get_country_code)
# remove rows with missing country codes
lci = lci.dropna(subset=["Country_Code"])
# taiwan is missing from lc-impact, add taiwan as new row with country code TW and asia averages
cf_annual_asia = 7.93972871304924E-16 # median for pasture in asia
row = pd.DataFrame({
    "Country": ["Taiwan"],
    "Pasture Median": [cf_annual_asia],
    "Country_Code": ["TW"],
})
lci = pd.concat([lci, row], ignore_index=True)
lci.set_index("Country_Code", inplace=True)
lci

Country code not found for  Azores
Country code not found for  Bonaire
Country code not found for  Canarias
Country code not found for  Cocos Islands
Country code not found for  Curacao
Country code not found for  Madeira
Country code not found for  Micronesia
Country code not found for  Palestinian Territory
Country code not found for  Saba
Country code not found for  Saint Barthelemy
Country code not found for  Saint Eustatius
Country code not found for  Saint Helena
Country code not found for  Saint Martin
Country code not found for  Sint Maarten
Country code not found for  South Georgia
Country code not found for  Vatican City


,Country,Pasture Median
Country_Code,,
AF,Afghanistan,4.042837e-16
AL,Albania,1.599513e-15
DZ,Algeria,1.643208e-16
AS,American Samoa,1.665278e-18
AD,Andorra,8.655631e-16
...,...,...
WF,Wallis and Futuna,1.311224e-14
YE,Yemen,7.550907e-16
ZM,Zambia,5.784345e-16


In [10]:
# find if some regions are missing from the lci 
missing_regions = [region for region in all_regions if region not in lci.index.unique()]
print("Missing regions (lc-impact): ", missing_regions)

# find out if some regions are missing from augmented exiobase regions 
lci_regions = lci.index
missing_from_all_regions = [region for region in lci_regions if region not in all_regions]
print("Missing regions (exiobase, augmented): ", missing_from_all_regions)

# assert that all regions in lci-water are in the all_regions list
assert sorted(lci_regions) == sorted(all_regions), "Some regions in lci-water are not in the all_regions list"

Missing regions (lc-impact):  []
Missing regions (exiobase, augmented):  []


In [11]:
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u = dr_s.copy()
dr_u = dr_u.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in all_row_region_keys:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in all_row_region_keys:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u = pd.concat([dr_u, combined_data])
dr_u

AT        \
                                                     Paddy rice Wheat   
AT Paddy rice                                               0.0   0.0   
   Wheat                                                    0.0   0.0   
   Cereal grains nec                                        0.0   0.0   
   Vegetables, fruit, nuts                                  0.0   0.0   
   Oil seeds                                                0.0   0.0   
...                                                         ...   ...   
YE Membership organisation services n.e.c. (91)             0.0   0.0   
   Recreational, cultural and sporting services (92)        0.0   0.0   
   Other services (93)                                      0.0   0.0   
   Private households with employed persons (95)            0.0   0.0   
   Extra-territorial organizations and bodies               0.0   0.0   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                      0.0   
   Wheat                                                           0.0   
   Cereal grains nec                                               0.0   
   Vegetables, fruit, nuts                                         0.0   
   Oil seeds                                                       0.0   
...                                                                ...   
YE Membership organisation services n.e.c. (91)                    0.0   
   Recreational, cultural and sporting services (92)               0.0   
   Other services (93)                                             0.0   
   Private households with employed persons (95)                   0.0   
   Extra-territorial organizations and bodies                      0.0   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                            0.0   
   Wheat                                                                 0.0   
   Cereal grains nec                                                     0.0   
   Vegetables, fruit, nuts                                               0.0   
   Oil seeds                                                             0.0   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                          0.0   
   Recreational, cultural and sporting services (92)                     0.0   
   Other services (93)                                                   0.0   
   Private households with employed persons (95)                         0.0   
   Extra-territorial organizations and bodies                            0.0   

                                                                \
                                                     Oil seeds   
AT Paddy rice                                              0.0   
   Wheat                                                   0.0   
   Cereal grains nec                                       0.0   
   Vegetables, fruit, nuts                                 0.0   
   Oil seeds                                               0.0   
...                                                        ...   
YE Membership organisation services n.e.c. (91)            0.0   
   Recreational, cultural and sporting services (92)       0.0   
   Other services (93)                                     0.0   
   Private households with employed persons (95)           0.0   
   Extra-territorial organizations and bodies              0.0   

                                                                             \
                                                     Sugar cane, sugar beet   
AT Paddy rice                                                           0.0   
   Wheat   

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [ ]:
# calculate the monetary impact factors from exiobase 2019 data
groups = exio3_11.satellite.get_index(
    as_dict=True,
    grouping_pattern=arguments["exiobase_grouping_patterns"]["land_pasture"]
)

exio3_19.satellite_agg = exio3_19.satellite.copy(new_name="Aggregated land stress - pasture")

for df_name, df in zip(exio3_19.satellite_agg.get_DataFrame(data=False, with_unit=True, with_population=False),
exio3_19.satellite_agg.get_DataFrame(data=True, with_unit=True, with_population=False)):
    if df_name == "unit":
        exio3_19.satellite_agg.__dict__[df_name] = df.groupby(groups).apply(lambda x: " & ".join(x.unit.unique()))
    else:
        exio3_19.satellite_agg.__dict__[df_name] = df.groupby(groups).sum()

exio_19_diag = exio3_19.satellite_agg.diag_stressor(("Land stress - pasture"))
exio3_19.exio_19_diag = exio_19_diag
exio3_19.calc_all()

In [13]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f = dr_u.copy()
blue_water_total = exio3_19.satellite_agg.M.loc["Land stress - pasture"]
scalars = blue_water_total.to_numpy() # multipliers for each column
print(len(scalars)) # 9800
print(dr_u.shape) # 37000 (impact region-sector pairs) * 9800 (consumption region-sector pairs)

# multiply each column of dr_u by the respective column value from exio3_19.satellite_agg.M
dr_f = dr_f * scalars # same as dr_f * diag(scalars) but more efficient with numpy broadcasting
dr_f

9800
(45800, 9800)


AT        \
                                                     Paddy rice Wheat   
AT Paddy rice                                               0.0   0.0   
   Wheat                                                    0.0   0.0   
   Cereal grains nec                                        0.0   0.0   
   Vegetables, fruit, nuts                                  0.0   0.0   
   Oil seeds                                                0.0   0.0   
...                                                         ...   ...   
YE Membership organisation services n.e.c. (91)             0.0   0.0   
   Recreational, cultural and sporting services (92)        0.0   0.0   
   Other services (93)                                      0.0   0.0   
   Private households with employed persons (95)            0.0   0.0   
   Extra-territorial organizations and bodies               0.0   0.0   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                      0.0   
   Wheat                                                           0.0   
   Cereal grains nec                                               0.0   
   Vegetables, fruit, nuts                                         0.0   
   Oil seeds                                                       0.0   
...                                                                ...   
YE Membership organisation services n.e.c. (91)                    0.0   
   Recreational, cultural and sporting services (92)               0.0   
   Other services (93)                                             0.0   
   Private households with employed persons (95)                   0.0   
   Extra-territorial organizations and bodies                      0.0   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                            0.0   
   Wheat                                                                 0.0   
   Cereal grains nec                                                     0.0   
   Vegetables, fruit, nuts                                               0.0   
   Oil seeds                                                             0.0   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                          0.0   
   Recreational, cultural and sporting services (92)                     0.0   
   Other services (93)                                                   0.0   
   Private households with employed persons (95)                         0.0   
   Extra-territorial organizations and bodies                            0.0   

                                                                \
                                                     Oil seeds   
AT Paddy rice                                              0.0   
   Wheat                                                   0.0   
   Cereal grains nec                                       0.0   
   Vegetables, fruit, nuts                                 0.0   
   Oil seeds                                               0.0   
...                                                        ...   
YE Membership organisation services n.e.c. (91)            0.0   
   Recreational, cultural and sporting services (92)       0.0   
   Other services (93)                                     0.0   
   Private households with employed persons (95)           0.0   
   Extra-territorial organizations and bodies              0.0   

                                                                             \
                                                     Sugar cane, sugar beet   
AT Paddy rice                                                           0.0   
   Wheat   

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [15]:
# sort rows on lci in same order as dr_f.index.sortlevel
lci = lci.reindex(dr_f.index.get_level_values(0).unique())

# build array from the relevent lci stressor
# every value should be repeated 200 times (number of sectors)
cf_average = lci["Pasture Median"].to_numpy()
cf_average = np.repeat(cf_average, 200) # 1D array of length 200 * number of regions in lci
print(cf_average.shape)
# expand cf_all_effects to match the shape of dr_f
cf_average = np.tile(cf_average, (dr_f.shape[1], 1)).T
print(cf_average.shape)

pdf = dr_f * cf_average
pdf

(45800,)
(45800, 9800)


AT        \
                                                     Paddy rice Wheat   
AT Paddy rice                                               0.0   0.0   
   Wheat                                                    0.0   0.0   
   Cereal grains nec                                        0.0   0.0   
   Vegetables, fruit, nuts                                  0.0   0.0   
   Oil seeds                                                0.0   0.0   
...                                                         ...   ...   
YE Membership organisation services n.e.c. (91)             0.0   0.0   
   Recreational, cultural and sporting services (92)        0.0   0.0   
   Other services (93)                                      0.0   0.0   
   Private households with employed persons (95)            0.0   0.0   
   Extra-territorial organizations and bodies               0.0   0.0   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                      0.0   
   Wheat                                                           0.0   
   Cereal grains nec                                               0.0   
   Vegetables, fruit, nuts                                         0.0   
   Oil seeds                                                       0.0   
...                                                                ...   
YE Membership organisation services n.e.c. (91)                    0.0   
   Recreational, cultural and sporting services (92)               0.0   
   Other services (93)                                             0.0   
   Private households with employed persons (95)                   0.0   
   Extra-territorial organizations and bodies                      0.0   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                            0.0   
   Wheat                                                                 0.0   
   Cereal grains nec                                                     0.0   
   Vegetables, fruit, nuts                                               0.0   
   Oil seeds                                                             0.0   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                          0.0   
   Recreational, cultural and sporting services (92)                     0.0   
   Other services (93)                                                   0.0   
   Private households with employed persons (95)                         0.0   
   Extra-territorial organizations and bodies                            0.0   

                                                                \
                                                     Oil seeds   
AT Paddy rice                                              0.0   
   Wheat                                                   0.0   
   Cereal grains nec                                       0.0   
   Vegetables, fruit, nuts                                 0.0   
   Oil seeds                                               0.0   
...                                                        ...   
YE Membership organisation services n.e.c. (91)            0.0   
   Recreational, cultural and sporting services (92)       0.0   
   Other services (93)                                     0.0   
   Private households with employed persons (95)           0.0   
   Extra-territorial organizations and bodies              0.0   

                                                                             \
                                                     Sugar cane, sugar beet   
AT Paddy rice                                                           0.0   
   Wheat   

In [16]:
# sum up all columns to get the total impact for each region-sector pair
pdf_total = pdf.sum()
pdf_total

AT  Paddy rice                                           0.000000e+00
    Wheat                                                1.360707e-16
    Cereal grains nec                                    1.305589e-16
    Vegetables, fruit, nuts                              8.937851e-17
    Oil seeds                                            1.244970e-16
                                                             ...     
WM  Membership organisation services n.e.c. (91)         7.193464e-16
    Recreational, cultural and sporting services (92)    3.719811e-15
    Other services (93)                                  7.328444e-15
    Private households with employed persons (95)        4.869002e-16
    Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [17]:
pd.DataFrame(pdf_total).to_csv("csv/pdf-land-pasture.csv", index=True)